# 📊 FiniexTestingIDE - Data Exploration
Interactive analysis of tick data using production data loader modules

In [ ]:
# Cell 1: Setup und Imports
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Projektpfad hinzufügen
sys.path.append('/app')

# Production Data Loader importieren
from python.data_worker.data_loader.core import TickDataLoader
from python.data_worker.data_loader.analytics import TickDataAnalyzer
from python.components.logger.bootstrap_logger import setup_logging

# Logging initialisieren (nur einmal)
vLog = setup_logging(name="DataExploration")

# Matplotlib defaults
plt.style.use('default')
sns.set_palette("husl")

print("✅ Alle Imports erfolgreich")
print(f"📂 Daten-Verzeichnis: {Path('/app/data/processed/').exists()}")

In [ ]:
# Cell 2: Data Loader initialisieren und Übersicht
loader = TickDataLoader('/app/data/processed/')
analyzer = TickDataAnalyzer(loader)

# Verfügbare Symbole anzeigen
symbols = loader.list_available_symbols()
print(f"🎯 Verfügbare Symbole ({len(symbols)}):")
print(f"   {', '.join(symbols)}")

# Daten-Zusammenfassung generieren
print("\n📊 Generiere Daten-Zusammenfassung...\n")
summary = analyzer.get_data_summary()

# Kompakte Übersicht
for symbol, info in summary.items():
    if 'error' not in info:
        print(f"{symbol:8s} | {info['total_ticks']:>10,} ticks | "
              f"{info['date_range']['duration']['days']:>3} days | "
              f"{info['file_size_mb']:>6.1f} MB | "
              f"{info['statistics']['tick_frequency_per_second']:>4.1f} t/s")

In [ ]:
# Cell 3: EURUSD Daten laden (OHNE Date-Filter = alle Daten)
print("📥 Lade EURUSD Daten...\n")
eurusd_data = loader.load_symbol_data("EURUSD")

print(f"\n✅ EURUSD Daten geladen:")
print(f"   Anzahl Ticks: {len(eurusd_data):,}")
print(f"   Zeitspanne:   {eurusd_data['timestamp'].min()} bis")
print(f"                 {eurusd_data['timestamp'].max()}")
print(f"   Spalten:      {len(eurusd_data.columns)}")

# Erste 5 Zeilen anzeigen
print("\n📋 Erste 5 Ticks:")
eurusd_data.head()

In [ ]:
# Cell 4: Basic Statistiken
print("📊 EURUSD Statistiken:\n")
print(f"   Bid Range:  {eurusd_data['bid'].min():.5f} - {eurusd_data['bid'].max():.5f}")
print(f"   Ask Range:  {eurusd_data['ask'].min():.5f} - {eurusd_data['ask'].max():.5f}")
print(f"   Ø Spread:   {eurusd_data['spread_points'].mean():.1f} Punkte")
print(f"   Ø Spread %: {eurusd_data['spread_pct'].mean():.4f}%")

# Beschreibende Statistiken
eurusd_data[['bid', 'ask', 'spread_points', 'spread_pct']].describe()

In [ ]:
# Cell 5: Preisbewegungen visualisieren
# Sampling für Performance (jeder 100. Tick)
sample_size = max(100, len(eurusd_data) // 1000)
sample_data = eurusd_data.iloc[::sample_size].copy()

fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# Subplot 1: Bid/Ask über Zeit
axes[0].plot(sample_data['timestamp'], sample_data['bid'], 
             label='Bid', alpha=0.7, color='#e74c3c', linewidth=1)
axes[0].plot(sample_data['timestamp'], sample_data['ask'], 
             label='Ask', alpha=0.7, color='#3498db', linewidth=1)
axes[0].set_title('EURUSD Bid/Ask Preis über Zeit', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Subplot 2: Spread in Punkten
axes[1].plot(sample_data['timestamp'], sample_data['spread_points'], 
             color='#2ecc71', alpha=0.7, linewidth=1)
axes[1].set_title('Spread in Punkten über Zeit', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

# Subplot 3: Spread als Prozent
axes[2].plot(sample_data['timestamp'], sample_data['spread_pct'], 
             color='#f39c12', alpha=0.7, linewidth=1)
axes[2].set_title('Spread als Prozent über Zeit', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"📈 Visualisierung: {len(sample_data):,} Datenpunkte ")
print(f"   (jeder {sample_size}. Tick von {len(eurusd_data):,} total)")

In [ ]:
# Cell 6: Spread-Analyse (Datenqualitäts-Check) - Optimiert
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Berechne sinnvolle Grenzen (99th percentile für bessere Darstellung)
spread_points_99 = eurusd_data['spread_points'].quantile(0.99)
spread_pct_99 = eurusd_data['spread_pct'].quantile(0.99)

# Subplot 1: Spread-Verteilung (Punkte) mit Limit
axes[0, 0].hist(eurusd_data['spread_points'], bins=50, range=(0, spread_points_99),
                alpha=0.7, color='#2ecc71', edgecolor='black')
axes[0, 0].set_title('Verteilung: Spread in Punkten (99% Range)', fontweight='bold')
axes[0, 0].set_xlabel('Spread (Punkte)')
axes[0, 0].set_ylabel('Häufigkeit')
axes[0, 0].grid(True, alpha=0.3)

# Annotiere Outlier-Info
n_outliers_points = len(eurusd_data[eurusd_data['spread_points'] > spread_points_99])
axes[0, 0].text(0.98, 0.97, f'{n_outliers_points:,} Outliers\n> {spread_points_99:.0f} Punkte',
                transform=axes[0, 0].transAxes, ha='right', va='top',
                fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Subplot 2: Spread-Prozent-Verteilung mit Limit
axes[0, 1].hist(eurusd_data['spread_pct'], bins=50, range=(0, spread_pct_99),
                alpha=0.7, color='#3498db', edgecolor='black')
axes[0, 1].set_title('Verteilung: Spread in Prozent (99% Range)', fontweight='bold')
axes[0, 1].set_xlabel('Spread (%)')
axes[0, 1].set_ylabel('Häufigkeit')
axes[0, 1].grid(True, alpha=0.3)

n_outliers_pct = len(eurusd_data[eurusd_data['spread_pct'] > spread_pct_99])
axes[0, 1].text(0.98, 0.97, f'{n_outliers_pct:,} Outliers\n> {spread_pct_99:.4f}%',
                transform=axes[0, 1].transAxes, ha='right', va='top',
                fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Subplot 3: Violin Plot statt Boxplot (zeigt Verteilung besser)
from matplotlib import pyplot as plt

# Violin Plot
parts = axes[1, 0].violinplot([box_data], positions=[1], showmeans=True, showmedians=True,
                               widths=0.7)

# Styling
for pc in parts['bodies']:
    pc.set_facecolor('#3498db')
    pc.set_alpha(0.7)

axes[1, 0].set_ylim(0, spread_points_99 * 1.2)
axes[1, 0].set_title('Spread-Verteilung (Violin Plot, 99% Range)', fontweight='bold')
axes[1, 0].set_ylabel('Spread (Punkte)')
axes[1, 0].set_xticks([1])
axes[1, 0].set_xticklabels(['EURUSD Spreads'])
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Statistik-Annotation
median_val = box_data.median()
mean_val = box_data.mean()
axes[1, 0].text(0.02, 0.98, 
                f'Median: {median_val:.1f}\nMean: {mean_val:.1f}\nIQR: {box_data.quantile(0.75)-box_data.quantile(0.25):.1f}',
                transform=axes[1, 0].transAxes, ha='left', va='top',
                fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

axes[1, 0].text(0.5, 0.95, f'{n_extreme:,} extreme Outliers\nabgeschnitten',
                transform=axes[1, 0].transAxes, ha='center', va='top',
                fontsize=8, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Subplot 4: Session-basierte Spread-Analyse
if 'session' in eurusd_data.columns:
    session_spreads = eurusd_data.groupby('session')['spread_points'].mean().sort_values()
    session_spreads.plot(kind='bar', alpha=0.7, ax=axes[1, 1], color='#e74c3c')
    axes[1, 1].set_title('Durchschnittlicher Spread pro Session', fontweight='bold')
    axes[1, 1].set_xlabel('Session')
    axes[1, 1].set_ylabel('Ø Spread (Punkte)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3, axis='y')
else:
    axes[1, 1].text(0.5, 0.5, 'Session-Daten\nnicht verfügbar',
                    ha='center', va='center', fontsize=12)
    axes[1, 1].set_title('Session-Analyse', fontweight='bold')

plt.tight_layout()
plt.show()

# Erweiterte Statistiken
print(f"📊 Spread-Statistiken (detailliert):\n")
print(f"   Min:         {eurusd_data['spread_points'].min():.1f} Punkte")
print(f"   25th Perc:   {eurusd_data['spread_points'].quantile(0.25):.1f} Punkte")
print(f"   Median:      {eurusd_data['spread_points'].median():.1f} Punkte")
print(f"   Mean:        {eurusd_data['spread_points'].mean():.1f} Punkte")
print(f"   75th Perc:   {eurusd_data['spread_points'].quantile(0.75):.1f} Punkte")
print(f"   99th Perc:   {spread_points_99:.1f} Punkte")
print(f"   Max:         {eurusd_data['spread_points'].max():.1f} Punkte")
print(f"\n   Outliers (>99th): {n_outliers_points:,} Ticks ({n_outliers_points/len(eurusd_data)*100:.2f}%)")

In [ ]:
# Cell 7: Multi-Symbol-Vergleich
# Lade nur die ersten 4 Symbole für Performance
symbols_to_compare = symbols[:4]  # Nimm die ersten 4 verfügbaren
colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

print(f"📊 Vergleiche {len(symbols_to_compare)} Symbole...\n")

for i, symbol in enumerate(symbols_to_compare):
    print(f"   Lade {symbol}...")
    data = loader.load_symbol_data(symbol)
    
    # Sampling für Performance
    sample_size = max(200, len(data) // 1000)
    sample = data.iloc[::sample_size]
    
    # Plot
    axes[i].plot(sample['timestamp'], sample['bid'], 
                color=colors[i], alpha=0.8, linewidth=1.5)
    axes[i].set_title(f'{symbol} Bid Price', fontsize=12, fontweight='bold')
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(True, alpha=0.3)
    
    # Y-Achse für bessere Vergleichbarkeit
    price_range = data['bid'].max() - data['bid'].min()
    mid_price = (data['bid'].max() + data['bid'].min()) / 2
    margin = price_range * 0.1
    axes[i].set_ylim(mid_price - price_range/2 - margin, 
                     mid_price + price_range/2 + margin)

plt.tight_layout()
plt.show()

print(f"\n✅ Preisbewegungen aller Symbole visualisiert")

In [ ]:
# Cell 8: Tick-Frequenz-Analyse
fig, ax = plt.subplots(figsize=(12, 6))

# Daten für Barplot sammeln
freq_data = []
for symbol in symbols:
    info = summary[symbol]
    if 'error' not in info:
        freq_data.append({
            'symbol': symbol,
            'freq': info['statistics']['tick_frequency_per_second']
        })

# Sortiert nach Frequenz
freq_data = sorted(freq_data, key=lambda x: x['freq'], reverse=True)

# Barplot
bars = ax.bar([d['symbol'] for d in freq_data], 
               [d['freq'] for d in freq_data], 
               alpha=0.7, color='#3498db', edgecolor='black')

# Werte auf Balken anzeigen
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}',
            ha='center', va='bottom', fontsize=9)

ax.set_title('Tick-Frequenz pro Symbol', fontsize=14, fontweight='bold')
ax.set_ylabel('Ticks pro Sekunde', fontsize=11)
ax.set_xlabel('Symbol', fontsize=11)
ax.tick_params(axis='x', rotation=45)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Datenqualitäts-Übersicht
print("\n📋 Datenqualitäts-Übersicht:\n")
print(f"{'Symbol':<10} {'Ticks':>12} {'Freq (t/s)':>12} {'Size (MB)':>12}")
print("=" * 50)
for d in freq_data:
    symbol = d['symbol']
    info = summary[symbol]
    print(f"{symbol:<10} {info['total_ticks']:>12,} "
          f"{d['freq']:>12.2f} {info['file_size_mb']:>12.1f}")

In [ ]:
# Cell 9: Intraday-Muster (Bonus: Stunden-basierte Analyse)
# Extrahiere Stunde aus Timestamp für EURUSD
eurusd_data['hour'] = pd.to_datetime(eurusd_data['timestamp']).dt.hour

# Gruppiere nach Stunde
hourly_stats = eurusd_data.groupby('hour').agg({
    'spread_points': 'mean',
    'bid': 'count'  # Anzahl Ticks
}).rename(columns={'bid': 'tick_count'})

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Tick-Aktivität über den Tag
axes[0].bar(hourly_stats.index, hourly_stats['tick_count'], 
            alpha=0.7, color='#3498db', edgecolor='black')
axes[0].set_title('EURUSD: Tick-Aktivität nach Stunde (UTC)', 
                  fontsize=12, fontweight='bold')
axes[0].set_xlabel('Stunde (UTC)')
axes[0].set_ylabel('Anzahl Ticks')
axes[0].grid(True, alpha=0.3, axis='y')

# Spread über den Tag
axes[1].plot(hourly_stats.index, hourly_stats['spread_points'], 
             marker='o', color='#2ecc71', linewidth=2, markersize=6)
axes[1].set_title('EURUSD: Durchschnittlicher Spread nach Stunde (UTC)', 
                  fontsize=12, fontweight='bold')
axes[1].set_xlabel('Stunde (UTC)')
axes[1].set_ylabel('Ø Spread (Punkte)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Trading-Sessions markieren
print("\n⏰ Trading Sessions (approximiert):")
print("   00-08 UTC: Sydney/Tokyo (Asia)")
print("   08-16 UTC: London (Europe)")
print("   16-00 UTC: New York (Americas)")

In [ ]:
# Cell 10: Finale Zusammenfassung
print("="*60)
print("📊 FINALE ZUSAMMENFASSUNG")
print("="*60)

total_ticks = sum(info['total_ticks'] for info in summary.values() if 'error' not in info)
total_size = sum(info['file_size_mb'] for info in summary.values() if 'error' not in info)

print(f"\n✅ Verfügbare Symbole:  {len(symbols)}")
print(f"📈 Gesamte Ticks:       {total_ticks:,}")
print(f"💾 Gesamtgröße:         {total_size:.1f} MB")
print(f"⚡ Ø Kompression:       ~10:1 (JSON → Parquet)")

print(f"\n🎯 Empfohlene Symbole für Testing:")
# Top 3 nach Tick-Count
top_symbols = sorted(
    [(s, info['total_ticks']) for s, info in summary.items() if 'error' not in info],
    key=lambda x: x[1], 
    reverse=True
)[:3]

for symbol, ticks in top_symbols:
    print(f"   • {symbol}: {ticks:,} ticks")

print(f"\n✨ Nächste Schritte:")
print(f"   1. Strategy Runner mit Batch-Testing")
print(f"   2. Worker-Performance-Analyse")
print(f"   3. Trade-Simulation mit Portfolio-Tracking")
print(f"\n" + "="*60)